# Working with Text Data and Naive Bayes in scikit-learn

## Agenda

**Working with text data**

- Representing text as data
- Reading SMS data
- Vectorizing SMS data
- Examining the tokens and their counts
- Bonus: Calculating the "spamminess" of each token

**Naive Bayes classification**

- Building a Naive Bayes model
- Comparing Naive Bayes with logistic regression

## Part 1: Representing text as data

From the [scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> Text Analysis is a major application field for machine learning algorithms. However the raw data, a sequence of symbols cannot be fed directly to the algorithms themselves as most of them expect **numerical feature vectors with a fixed size** rather than the **raw text documents with variable length**.

We will use [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) to "convert text into a matrix of token counts":

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# start with a simple example
simple_train = ['call you tonight', 'Call me a cab', 'please call me... PLEASE!']

In [ ]:
# learn the 'vocabulary' of the training data
vect = CountVectorizer()
vect.fit(simple_train)
vect.get_feature_names()

In [ ]:
# transform training data into a 'document-term matrix'
simple_train_dtm = vect.transform(simple_train)
simple_train_dtm

In [ ]:
# print the sparse matrix
print(simple_train_dtm)

In [ ]:
# convert sparse matrix to a dense matrix
simple_train_dtm.toarray()

In [ ]:
# examine the vocabulary and document-term matrix together
import pandas as pd
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

In [ ]:
# create a document-term matrix on your own
simple_train = ["Sorry, Ill call later", 
                "K Did you call me just now ah", 
                "I call you later, don't have network. If urgnt, sms me"]

In [ ]:
#complete your work below
# learn the 'vocabulary' of the training data
vect = CountVectorizer()
vect.fit(simple_train)
vect.get_feature_names()

In [ ]:
# transform training data into a 'document-term matrix'
simple_train_dtm = vect.transform(simple_train)
simple_train_dtm

In [ ]:
# examine the vocabulary and document-term matrix together
import pandas as pd
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

From the [scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> In this scheme, features and samples are defined as follows:

> - Each individual token occurrence frequency (normalized or not) is treated as a **feature**.
> - The vector of all the token frequencies for a given document is considered a multivariate **sample**.

> A **corpus of documents** can thus be represented by a matrix with **one row per document** and **one column per token** (e.g. word) occurring in the corpus.

> We call **vectorization** the general process of turning a collection of text documents into numerical feature vectors. This specific strategy (tokenization, counting and normalization) is called the **Bag of Words** or "Bag of n-grams" representation. Documents are described by word occurrences while completely ignoring the relative position information of the words in the document.

In [ ]:
# transform testing data into a document-term matrix (using existing vocabulary)
simple_test = ["please don't call me"]
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm.toarray()

In [ ]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names())

**Summary:**

- `vect.fit(train)` learns the vocabulary of the training data
- `vect.transform(train)` uses the fitted vocabulary to build a document-term matrix from the training data
- `vect.transform(test)` uses the fitted vocabulary to build a document-term matrix from the testing data (and ignores tokens it hasn't seen before)

## Part 2: Reading SMS data

In [ ]:
# read tab-separated file
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/sms.tsv'
col_names = ['label', 'message']
sms = pd.read_table(url, sep='\t', header=None, names=col_names)
print(sms.shape)

In [ ]:
# Visualize the data in some fashion so you can see how it's formed
sms.head(20)

In [ ]:
# How many ham vs spam are there? Remember how to count values
sms.label.value_counts()

In [ ]:
# convert label to a numeric variable - use the map function
sms['label'] = sms.label.map({'ham':0, 'spam':1})

#check your work
sms.head()

In [ ]:
# define X and y so we can use them for analysis later. What is our feature (X), vs what is our response variable (y)
X = sms.message
y = sms.label

In [ ]:
# split into training and testing sets - Use a random_state of 1 to keep us all consistent
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

#How big are your test and train sets?
print(X_train.shape)
print(X_test.shape)

## Part 3: Vectorizing SMS data

In [ ]:
# instantiate the vectorizer
vect = CountVectorizer()

In [ ]:
# learn training data vocabulary, then create document-term matrix
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

In [ ]:
# alternative: combine fit and transform into a single step
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm

In [ ]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

## Part 4: Examining the tokens and their counts

In [ ]:
# store token names
X_train_tokens = vect.get_feature_names()

In [ ]:
# first 50 tokens
print(X_train_tokens[:50])

In [ ]:
# last 50 tokens
print(X_train_tokens[-50:])

In [ ]:
# view X_train_dtm as a dense matrix
X_train_dtm.toarray()

In [ ]:
# count how many times EACH token appears across ALL messages in X_train_dtm
import numpy as np
X_train_counts = np.sum(X_train_dtm.toarray(), axis=0)
X_train_counts

In [ ]:
X_train_counts.shape

In [ ]:
# create a DataFrame of tokens with their counts
pd.DataFrame({'token':X_train_tokens, 'count':X_train_counts}).sort_values('count')

## Bonus: Calculating the "spamminess" of each token

In [ ]:
# create separate DataFrames for ham and spam
sms_ham = sms[sms.label==0]
sms_spam = sms[sms.label==1]

In [ ]:
# learn the vocabulary of ALL messages and save it
# You're going to need to fit data... but what data is fully comprehensive of both spam and ham?
vect.fit(sms.message)

# Go ahead and capture your feature names at the same time. We'll use the name 'all_tokens'
all_tokens = vect.get_feature_names()

In [ ]:
# create document-term matrices for ham and spam
ham_dtm = vect.transform(sms_ham.message)
spam_dtm = vect.transform(sms_spam.message)

In [ ]:
# count how many times EACH token appears across ALL ham messages
ham_counts = np.sum(ham_dtm.toarray(), axis=0)

In [ ]:
# count how many times EACH token appears across ALL spam messages
spam_counts = np.sum(spam_dtm.toarray(), axis=0)

In [ ]:
# create a DataFrame of tokens with their separate ham and spam counts
token_counts = pd.DataFrame({'token':all_tokens, 'ham':ham_counts, 'spam':spam_counts})

In [ ]:
# add one to ham and spam counts to avoid dividing by zero (in the step that follows)
token_counts['ham'] = token_counts.ham + 1
token_counts['spam'] = token_counts.spam + 1

In [ ]:
# calculate ratio of spam-to-ham for each token
token_counts['spam_ratio'] = token_counts.spam / token_counts.ham
token_counts.sort_values('spam_ratio' ,ascending=False)

In [ ]:
#observe spam messages that contain the word 'claim'
claim_messages = sms.message[sms.message.str.contains('claim')]


for message in claim_messages[0:5]:
  print(message, '\n')

## Part 5: Building a Naive Bayes model

We will use [Multinomial Naive Bayes](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html):

> The multinomial Naive Bayes classifier is suitable for classification with **discrete features** (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.

In [ ]:
# train a Naive Bayes model using X_train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
%time nb.fit(X_train_dtm, y_train)

In [ ]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [ ]:
# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

In [ ]:
# confusion matrix
print(metrics.confusion_matrix(y_test, y_pred_class))

In [ ]:
# predict probabilities
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

In [ ]:
# calculate AUC
print(metrics.roc_auc_score(y_test, y_pred_prob))

In [ ]:
# print message text for the false positives
X_test[y_test < y_pred_class]

In [ ]:
# print message text for the false negatives
X_test[y_test > y_pred_class]

In [ ]:
# what do you notice about the false negatives?
X_test[3132]


# Come up with some thoughts with your table


## Part 6: Comparing Naive Bayes with logistic regression

In [ ]:
#Create a logitic regression
# import/instantiate/fit
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
%time logreg.fit(X_train_dtm, y_train)

In [ ]:
# class predictions and predicted probabilities

y_pred_class = logreg.predict(X_test_dtm)
print(metrics.accuracy_score(y_test, y_pred_class))

y_pred_prob = logreg.predict_proba(X_test_dtm)[:,1]
print(y_pred_prob)

In [ ]:
# calculate accuracy and AUC
print('Accuracy', metrics.accuracy_score(y_test, y_pred_class))
print('AUC Score',metrics.roc_auc_score(y_test, y_pred_prob))